In [1]:
import numpy as np
import pandas as pd
import glob
import os
from importlib import reload
from sklearn.metrics.pairwise import haversine_distances

In [2]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", module = "matplotlib\..*" )

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import colors

In [3]:
filename = '../data/geonames_countryInfo.txt'
meta = pd.read_csv(filename, skiprows=49, usecols=[0,1,4,5,6,7,8,9], keep_default_na=False)
isos = pd.unique(meta['ISO'])

meta.head()

,ISO,ISO3,Country,Capital,Area(in sq km),Population,Continent,GlobalDesignation
0,AD,AND,Andorra,Andorra la Vella,468.0,77006,EU,GN
1,AE,ARE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,GS
2,AF,AFG,Afghanistan,Kabul,647500.0,37172386,AS,GS
3,AG,ATG,Antigua and Barbuda,St. John's,443.0,96286,NA,GS
4,AI,AIA,Anguilla,The Valley,102.0,13254,NA,GS


In [4]:
isos

array(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR', 'AS',
       'AT', 'AU', 'AW', 'AX', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG',
       'BH', 'BI', 'BJ', 'BL', 'BM', 'BN', 'BO', 'BQ', 'BR', 'BS', 'BT',
       'BV', 'BW', 'BY', 'BZ', 'CA', 'CC', 'CD', 'CF', 'CG', 'CH', 'CI',
       'CK', 'CL', 'CM', 'CN', 'CO', 'CR', 'CU', 'CV', 'CW', 'CX', 'CY',
       'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'EH',
       'ER', 'ES', 'ET', 'FI', 'FJ', 'FK', 'FM', 'FO', 'FR', 'GA', 'GB',
       'GD', 'GE', 'GF', 'GG', 'GH', 'GI', 'GL', 'GM', 'GN', 'GP', 'GQ',
       'GR', 'GS', 'GT', 'GU', 'GW', 'GY', 'HK', 'HM', 'HN', 'HR', 'HT',
       'HU', 'ID', 'IE', 'IL', 'IM', 'IN', 'IO', 'IQ', 'IR', 'IS', 'IT',
       'JE', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KP',
       'KR', 'XK', 'KW', 'KY', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR',
       'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MF', 'MG',
       'MH', 'MK', 'ML', 'MM', 'MN', 'MO', 'MP', 'M

In [5]:
filename = '../data/geonames_cities500.txt'
popd = pd.read_table(filename, header=None, dtype={10:'str', 13:'str'}, keep_default_na=False)
popdcolumns = ['geonameid', 'name', 'asciiname', 'alternatenames',
               'latitude', 'longitude', 'feature_class', 'feature_code',
               'country_code', 'cc2', 'admin1_code', 'admin2_code', 'admin3_code',
               'admin4_code', 'population', 'elevation','dem', 'timezone', 'modification_date']
popd.columns = popdcolumns
popd = popd.loc[:, ['asciiname', 'country_code', 'population', 'latitude', 'longitude']]

popd.head()

,asciiname,country_code,population,latitude,longitude
0,Andorra la Vella,AD,20430,42.50779,1.52109
1,Arinsal,AD,1419,42.57205,1.48453
2,Canillo,AD,3292,42.56760,1.59756
3,El Tarter,AD,1052,42.57952,1.65362
4,Encamp,AD,11223,42.53474,1.58014


In [6]:
iso = 'CY'
dfcountry = popd[popd['country_code'] == iso].sort_values(by='population', ascending=False)
#dfcountry = dfcountry[dfcountry['population'] > 100]
print(dfcountry.shape)

if len(dfcountry) > 0:
    locs = np.deg2rad(dfcountry.iloc[:, 3:5].values)
    dists = haversine_distances(locs)
    dists *= 6371000/1000

dfcountry

(122, 5)


,asciiname,country_code,population,latitude,longitude
37083,Nicosia,CY,200452,35.17531,33.36420
37065,Limassol,CY,154000,34.68406,33.03794
37060,Larnaca,CY,72000,34.92291,33.62330
37115,Strovolos,CY,67904,35.14867,33.33384
37035,Famagusta,CY,42526,35.12489,33.94135
...,...,...,...,...,...
37051,Koloni,CY,264,34.74858,32.46565
37074,Meladeia,CY,20,34.98816,32.50665
37120,Troodos,CY,0,34.92344,32.88050
37123,Tsada - Tremithousa - Tala (Borders),CY,0,34.83430,32.45363


In [7]:
i = 0
print(dfcountry.iloc[i, 0])
for j in range(min(len(dfcountry),25)):
    print('\t', j, '\t', dfcountry.iloc[j, 0], '\t', np.round(dists[i,j],2) )

Nicosia
	 0 	 Nicosia 	 0.0
	 1 	 Limassol 	 62.2
	 2 	 Larnaca 	 36.66
	 3 	 Strovolos 	 4.05
	 4 	 Famagusta 	 52.77
	 5 	 Paphos 	 96.4
	 6 	 Kyrenia 	 18.41
	 7 	 Protaras 	 65.7
	 8 	 Pergamos 	 34.65
	 9 	 Morfou 	 33.8
	 10 	 Aradippou 	 32.49
	 11 	 Paralimni 	 58.18
	 12 	 Geroskipou 	 95.32
	 13 	 Lefka 	 47.28
	 14 	 Geri 	 9.25
	 15 	 Ypsonas 	 65.43
	 16 	 Dali 	 17.74
	 17 	 Xylofagou 	 49.29
	 18 	 Tseri 	 11.96
	 19 	 Livadia 	 34.25
	 20 	 Dhromolaxia 	 39.02
	 21 	 Lapithos 	 25.05
	 22 	 Rizokarpaso 	 103.28
	 23 	 Deryneia 	 55.64
	 24 	 Emba 	 94.93


In [19]:
bar = np.argsort(dists[i])
foo = dfcountry.iloc[bar, :].values
for j in range(20):
    print(j, foo[j,[0,2]], np.round(dists[i,bar[j]],2), sep='\t')

0	['Nicosia' 200452]	0.0
1	['Strovolos' 67904]	4.05
2	['Geri' 7639]	9.25
3	['Tseri' 5498]	11.96
4	['Kato Deftera' 1706]	12.89
5	['Pano Deftera' 1983]	14.06
6	['Mammari' 1122]	14.53
7	['Kokkinotrimithia' 3223]	15.16
8	['Psimolofou' 1686]	15.59
9	['Ergates' 1656]	17.46
10	['Dali' 6085]	17.74
11	['Episkopeio' 533]	18.27
12	['Kyrenia' 26701]	18.41
13	['Pera' 1060]	18.68
14	['Athienou' 4444]	20.49
15	['Arediou' 1129]	20.8
16	['Alampra' 1192]	20.96
17	['Meniko' 1023]	21.26
18	['Lympia' 2262]	21.53
19	['Akaki' 2769]	21.91


In [66]:
tol = 16.1
corresp = np.column_stack([np.ones(len(dfcountry)).astype(int), 
                           np.arange(len(dfcountry)), 
                           dfcountry.population.values])

for idx in range(1, len(corresp)):
    for j in range(idx):
        if dists[idx, j] < tol:
            k = corresp[j, 1]
            corresp[idx, 0] = 0
            corresp[idx,1] = k
            corresp[k,2] += corresp[idx, 2]
            
            print(idx,'\t',dfcountry.iloc[idx,0], '-->', dfcountry.iloc[k,0], '\t(', np.around(dists[idx,k],1), ')')
            break

8 	 Katutura --> Windhoek 	( 4.7 )
19 	 Ongwediva --> Oshakati 	( 6.6 )
37 	 Ongandjera --> Okahao 	( 0.5 )


In [67]:
mask = np.nonzero(corresp[:,0])[0]
dfred = dfcountry.iloc[mask,:]
dfred = dfred.assign(population=corresp[mask, 2])
dfred = dfred.sort_values(by='population', ascending=False)
dfred = dfred.assign(incountry_rank=np.arange(len(dfred)))
print(dfred.shape)
dfred.head(15)

(42, 6)


,asciiname,country_code,population,latitude,longitude,incountry_rank
135516,Windhoek,NA,289375,-22.55941,17.08323,0
135522,Rundu,NA,58172,-17.91796,19.77314,1
135557,Walvis Bay,NA,52058,-22.95750,14.50528,2
135529,Oshakati,NA,43272,-17.78833,15.70436,3
135521,Swakopmund,NA,25047,-22.67842,14.52663,4
135514,Katima Mulilo,NA,25027,-17.50467,24.27574,5
135551,Grootfontein,NA,24099,-19.56667,18.11667,6
135523,Rehoboth,NA,21377,-23.31700,17.09000,7
135525,Otjiwarongo,NA,21224,-20.46369,16.64772,8
135539,Okahandja,NA,20879,-21.98333,16.91667,9


# Concatenating everything

In [68]:
tol = 16.1
countrydf = []

for iso in isos:
    dfcountry = popd[popd['country_code'] == iso].sort_values(by='population', ascending=False)
    dfcountry = dfcountry[dfcountry['population'] > 100]

    if len(dfcountry) > 0:
        locs = np.deg2rad(dfcountry.iloc[:, 3:].values)
        dists = haversine_distances(locs)
        dists *= 6371000/1000
    
        corresp = np.column_stack([np.ones(len(dfcountry)).astype(int), 
                                   np.arange(len(dfcountry)), 
                                   dfcountry.population.values])

        for idx in range(1, len(corresp)):
            for j in range(idx):
                if dists[idx, j] < tol:
                    k = corresp[j, 1]
                    corresp[idx, 0] = 0
                    corresp[idx,1] = k
                    corresp[k,2] += corresp[idx, 2]
                    break
        mask = np.nonzero(corresp[:,0])[0]
        dfred = dfcountry.iloc[mask,:]
        dfred = dfred.assign(population=corresp[mask, 2])
        dfred = dfred.sort_values(by='population', ascending=False)
        dfred = dfred.assign(incountry_rank=np.arange(len(dfred)))

        countrydf.append(dfred)
    
    print(iso)

AD
AE
AF
AG
AI
AL
AM
AO
AQ
AR
AS
AT
AU
AW
AX
AZ
BA
BB
BD
BE
BF
BG
BH
BI
BJ
BL
BM
BN
BO
BQ
BR
BS
BT
BV
BW
BY
BZ
CA
CC
CD
CF
CG
CH
CI
CK
CL
CM
CN
CO
CR
CU
CV
CW
CX
CY
CZ
DE
DJ
DK
DM
DO
DZ
EC
EE
EG
EH
ER
ES
ET
FI
FJ
FK
FM
FO
FR
GA
GB
GD
GE
GF
GG
GH
GI
GL
GM
GN
GP
GQ
GR
GS
GT
GU
GW
GY
HK
HM
HN
HR
HT
HU
ID
IE
IL
IM
IN
IO
IQ
IR
IS
IT
JE
JM
JO
JP
KE
KG
KH
KI
KM
KN
KP
KR
XK
KW
KY
KZ
LA
LB
LC
LI
LK
LR
LS
LT
LU
LV
LY
MA
MC
MD
ME
MF
MG
MH
MK
ML
MM
MN
MO
MP
MQ
MR
MS
MT
MU
MV
MW
MX
MY
MZ
NA
NC
NE
NF
NG
NI
NL
NO
NP
NR
NU
NZ
OM
PA
PE
PF
PG
PH
PK
PL
PM
PN
PR
PS
PT
PW
PY
QA
RE
RO
RS
RU
RW
SA
SB
SC
SD
SS
SE
SG
SH
SI
SJ
SK
SL
SM
SN
SO
SR
ST
SV
SX
SY
SZ
TC
TD
TF
TG
TH
TJ
TK
TL
TM
TN
TO
TR
TT
TV
TW
TZ
UA
UG
UM
US
UY
UZ
VA
VC
VE
VG
VI
VN
VU
WF
WS
YE
YT
ZA
ZM
ZW
CS
AN


In [69]:
dfred = pd.concat(countrydf, ignore_index=True)
print(dfred.shape)

filename = '../data/merged_cities500.csv'
dfred.to_csv(filename, index=False)
dfred.head(10)

(33554, 6)


,asciiname,country_code,population,latitude,longitude,incountry_rank
0,Andorra la Vella,AD,75127,42.50779,1.52109,0
1,Dubai,AE,3478300,25.07725,55.30927,0
2,Sharjah,AE,1814508,25.33737,55.41206,1
3,Abu Dhabi,AE,1074415,24.45118,54.39696,2
4,Ras Al Khaimah City,AE,351943,25.78953,55.94320,3
5,Al Fujairah City,AE,168822,25.11641,56.34141,4
6,Zayed City,AE,63482,23.65416,53.70522,5
7,Umm Al Quwain City,AE,62747,25.56473,55.55517,6
8,Dibba Al-Fujairah,AE,56395,25.59246,56.26176,7
9,Al Ain City,AE,55091,24.19167,55.76056,8


# Connection to research cities

In [75]:
filename = '../data/merged_cities500.csv'
popg = pd.read_csv(filename,keep_default_na=False)

In [4]:
src = '../results/'
dst = '../plots/'

filename = src + 'merged_locations.csv'
auth = pd.read_csv(filename)

print(auth.shape)
countries = pd.unique(auth.country)

(7972, 6)


In [54]:
country=countries[122]
iso = meta[meta['Country'] == country]['ISO'].values[0]
print(country, iso)

Myanmar MM


In [49]:
resc = auth[auth['country'] == country]
resloc = resc.loc[:, ['latitude', 'longitude']].values
resloc = np.deg2rad(resloc)
print(resc.shape)
resc.head()

(7, 6)


,incountry_rank,country,location,score,latitude,longitude
4268,0,Namibia,"Windhoek, Namibia",60,-22.560881,17.065755
4269,1,Namibia,"Oshakati, Namibia",3,-17.789453,15.705779
4270,2,Namibia,"Maltahohe, Namibia",2,-24.838059,16.980135
4271,3,Namibia,"Henties Bay, Namibia",2,-22.113496,14.283204
4272,4,Namibia,"Swakopmund, Namibia",1,-22.646743,14.600491


In [77]:
popc = popg[popg['country_code'] == 'NA']
poploc = popc.loc[:, ['latitude', 'longitude']].values
poploc = np.deg2rad(poploc)
print(popc.shape)
popc

(40, 6)


,asciiname,country_code,population,latitude,longitude,incountry_rank
17903,Windhoek,NA,289375,-22.55941,17.08323,0
17904,Rundu,NA,58172,-17.91796,19.77314,1
17905,Walvis Bay,NA,52058,-22.95750,14.50528,2
17906,Oshakati,NA,43272,-17.78833,15.70436,3
17907,Swakopmund,NA,25047,-22.67842,14.52663,4
17908,Katima Mulilo,NA,25027,-17.50467,24.27574,5
17909,Grootfontein,NA,24099,-19.56667,18.11667,6
17910,Rehoboth,NA,21377,-23.31700,17.09000,7
17911,Otjiwarongo,NA,21224,-20.46369,16.64772,8
17912,Okahandja,NA,20879,-21.98333,16.91667,9


In [60]:
popg[popg['asciiname'] == 'Windhoek']

,asciiname,country_code,population,latitude,longitude,incountry_rank


In [10]:
dists = haversine_distances(resloc, poploc)
dists *= 6371000/1000

In [29]:
i = 0
for i in range(25):
    argmatch = np.argmin(dists[i])
    match = popc.iloc[argmatch, 0]
    print(i, resc.iloc[i,2], '-->', match, popc.iloc[argmatch, 2], sep='\t')

0	Mexico City, CDMX, Mexico	-->	Mexico City	29330422
1	Cuernavaca, Morelos, Mexico	-->	Cuernavaca	953690
2	Texcoco, Estado De Mexic, Mexico	-->	Ecatepec de Morelos	3830227
3	Merida, Yucatan, Mexico	-->	Merida	938429
4	Irapuato, Guanajuato, Mexico	-->	Irapuato	509573
5	Morelia, Michoacan, Mexico	-->	Morelia	752464
6	Xalapa, Veracruz, Mexico	-->	Xalapa de Enriquez	769614
7	San Luis Potosi, San Luis Potosi, Mexico	-->	San Luis Potosi	1031279
8	Saltillo, Coahuila, Mexico	-->	Saltillo	785174
9	Zapopan, Jalisco, Mexico	-->	Guadalajara	4171299
10	Hermosillo, Sonora, Mexico	-->	Hermosillo	718855
11	Queretaro, Queretaro, Mexico	-->	Santiago de Queretaro	984394
12	Monterrey, NL, Mexico	-->	Monterrey	3757660
13	La Paz, Baja California Sur, Mexico	-->	La Paz	222052
14	Culiacan, Sinaloa, Mexico	-->	Culiacan	698533
15	Celaya, Guanajuato, Mexico	-->	Celaya	643695
16	Yautepec, Morelos, Mexico	-->	Cuautla	418212
17	Puebla, Puebla, Mexico	-->	Puebla	2113580
18	Toluca, Estado De Mexic, Mexico	-->	Toluca	

In [30]:
i = 12
argmatch = np.argsort(dists[i])[:5]
print(resc.iloc[i,2], ':')
for j in argmatch:
    print('\t', popc.iloc[j, 0], '\t', np.round(dists[i,j],2))

Monterrey, NL, Mexico :
	 Monterrey 	 1.31
	 Ciudad Benito Juarez 	 22.51
	 Garcia 	 31.55
	 Santiago 	 33.39
	 Hidalgo 	 34.46


In [38]:
matchs = ['' for i in range(len(dists))]

population = np.zeros(len(dists), dtype=int)
diffs = np.zeros(len(dists))

for i in range(len(dists)):
    argmatch = np.argmin(dists[i])
    match = popc.iloc[argmatch, 0]
    
    matchs[i] = match
    population[i] = popc.iloc[argmatch, 2]
    diffs[i] = dists[i, argmatch]

In [42]:
cdata = auth[auth['country'] == country].copy()
cdata['matched_pop'] = matchs
cdata['dist_difference'] = diffs
cdata['population'] = population
cdata.head()

,incountry_rank,country,location,score,latitude,longitude,matched_pop,dist_difference,population
4030,0,Mexico,"Mexico City, CDMX, Mexico",2911,19.432608,-99.133208,Mexico City,0.741717,29330422
4031,1,Mexico,"Cuernavaca, Morelos, Mexico",703,18.924209,-99.221566,Cuernavaca,0.988629,953690
4032,2,Mexico,"Texcoco, Estado De Mexic, Mexico",697,19.506038,-98.883150,Ecatepec de Morelos,21.604671,3830227
4033,3,Mexico,"Merida, Yucatan, Mexico",546,20.967370,-89.592586,Merida,2.682547,938429
4034,4,Mexico,"Irapuato, Guanajuato, Mexico",497,20.678665,-101.354496,Irapuato,0.282456,509573


# Concatenating all countries

In [84]:
cdata = []

for i in range(len(countries)):
    country=countries[i]
    if country in ['England', 'North Ireland', 'Scotland', 'Wales']:
        iso = 'GB'
    else:
        iso = meta[meta['Country'] == country]['ISO'].values[0]
        
    resc = auth[auth['country'] == country].copy()
    resloc = resc.loc[:, ['latitude', 'longitude']].values
    resloc = np.deg2rad(resloc)
    
    popc = popg[popg['country_code'] == iso]
    poploc = popc.loc[:, ['latitude', 'longitude']].values
    poploc = np.deg2rad(poploc)
    
    dists = haversine_distances(resloc, poploc)
    dists *= 6371000/1000
    
    matchs = ['' for j in range(len(dists))]
    population = np.zeros(len(dists), dtype=int)
    poprank = np.zeros_like(population)
    diffs = np.zeros(len(dists))

    for j in range(len(dists)):
        argmatch = np.argmin(dists[j])
        match = popc.iloc[argmatch, 0]

        matchs[j] = match
        population[j] = popc.iloc[argmatch, 2]
        poprank[j] = popc.iloc[argmatch, 5]
        diffs[j] = dists[j, argmatch]
    
    resc['country_code'] = iso
    resc['matched_pop'] = matchs
    resc['dist_difference'] = diffs
    resc['population'] = population
    resc['incountry_pop_rank'] = poprank
    
    cdata.append(resc)

In [85]:
dfred = pd.concat(cdata, ignore_index=True)
print(dfred.shape)

(7972, 11)


In [90]:
filename = '../data/merged_research_cities500.csv'
dfred = dfred.iloc[:, [6,1,2,7,8,3,9,0,10,4,5]]
dfred.to_csv(filename, index=False)

In [91]:
dfred.head(10)

,country_code,country,location,matched_pop,dist_difference,score,population,incountry_rank,incountry_pop_rank,latitude,longitude
0,AF,Afghanistan,"Kabul, Afghanistan",Kabul,4.419195,7,4434550,0,0,34.555349,69.207486
1,AF,Afghanistan,"Kandahar, Afghanistan",Kandahar,3.089936,3,523300,1,3,31.628871,65.737175
2,AF,Afghanistan,"Helmand, Afghanistan",Markaz-e Hukumat-e Darweshan,34.057000,2,9012,2,70,31.363647,63.958611
3,AF,Afghanistan,"Herat, Afghanistan",Herat,0.657768,1,586235,3,1,34.352865,62.204029
4,AF,Afghanistan,"Logar, Afghanistan",Baraki Barak,23.024792,1,35552,4,31,34.014552,69.192392
5,AF,Afghanistan,"Bamyan, Afghanistan",Bamyan,1.401247,1,61863,5,18,34.810007,67.821210
6,AL,Albania,"Tirana, Albania",Tirana,0.016809,45,390800,0,0,41.327546,19.818698
7,AL,Albania,"Vlore, Albania",Vlore,0.746803,1,99062,1,4,40.466067,19.491356
8,AL,Albania,"Peja, Albania",Kukes,65.672168,1,17832,2,16,42.659287,20.288736
9,DZ,Algeria,"Algiers, Algeria",Algiers,3.502340,275,3073716,0,0,36.753768,3.058756


In [7]:
dfred[dfred['population'] < 1]

NameError: name 'dfred' is not defined